In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
import lightgbm as lgb
import types

We all love the power of learning libraries like Keras, XGBoost or LightGBM, and we all love SkLearn for offering a unified API for many different tools we use as well as augmenting them with a set of additional tools like metrics, CV, feature selection etc. However, sometimes your favourite machine learning library might not play nicely with SkLearn. LightGBM supports categorical variables, but SkLearn mostly does not. Many tree-based libraries can utilise missing values natively, but SkLearn whines about NaNs all the time. Keras may need some additional tweaking on that `fit()` call via parameters, but so many SkLearn interfaces restrict `fit()` to take only X and y... However, we are not helpless. While monkey patching is often frowned upon in more serious settings, this is where it can really help us. Here are some examples to make SkLearn work nicely with LightGBM, but the same principle applies to any library.

## 1. Stop SkLearn from complaining about NaN or Inf

Suppose you want to use the fancy automatic recursive feature elimination in SkLearn for your LightGBM classifier. So as usual, you will set up the code:
```
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1111)
clf_params = {
    ...
    }
clf = lgb.LGBMClassifier(**clf_params)
cv = RFECV(clf, step=5, min_features_to_select=50, cv=folds, verbose=10,
               scoring=make_scorer(wloss, greater_is_better=False, needs_proba=True))
cv.fit(X, y)               
```
But then you are immediately told you cannot do so because SkLearn checked your inputs and there are missing values or infinite values. Looking through the trace stack, you can pinpoint the issue to a call to `check_X_y()` or `check_array()`. Looking at the definition of these functions, you will notice that they have a parameter called `force_all_finite`. Here is the documentation:
```
force_all_finite : boolean or 'allow-nan', (default=True)
        Whether to raise an error on np.inf and np.nan in X. This parameter
        does not influence whether y can have np.inf or np.nan values.
        The possibilities are:

        - True: Force all values of X to be finite.
        - False: accept both np.inf and np.nan in X.
        - 'allow-nan':  accept  only  np.nan  values in  X.  Values  cannot  be
          infinite.
```
So we can easily solve this problem with setting it to `False`, right? Unfortunately we don't call this function, but SkLearn does. We can alter the SkLearn library source code, but that usually is a bad idea. What we can do is to replace that one method or one module. In this case, we can simply make a copy of the whole `rfe` module and add `force_all_finite=True` to the `check_X_y()` and `check_array()` calls. The modified code this hidden below. It can be saved to a separate file for importing the necessary classes instead of from OG SkLearn. You may have to change all `..` in imports at the beginning of the file to `sklearn.`.

In [ ]:
import numpy as np
from sklearn.utils import check_X_y, safe_sqr
from sklearn.utils.metaestimators import if_delegate_has_method
from sklearn.utils.metaestimators import _safe_split
from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator
from sklearn.base import MetaEstimatorMixin
from sklearn.base import clone
from sklearn.base import is_classifier
from sklearn.utils import Parallel, delayed, effective_n_jobs
from sklearn.model_selection import check_cv
from sklearn.model_selection._validation import _score
from sklearn.metrics.scorer import check_scoring
from sklearn.feature_selection.base import SelectorMixin

from sklearn.utils import check_array, safe_mask
from warnings import warn


def _rfe_single_fit(rfe, estimator, X, y, train, test, scorer):
    """
    Return the score for a fit across one fold.
    """
    X_train, y_train = _safe_split(estimator, X, y, train)
    X_test, y_test = _safe_split(estimator, X, y, test, train)
    return rfe._fit(
        X_train, y_train, lambda estimator, features:
        _score(estimator, X_test[:, features], y_test, scorer)).scores_


class RFE(BaseEstimator, MetaEstimatorMixin, SelectorMixin):
    """Feature ranking with recursive feature elimination.

    Given an external estimator that assigns weights to features (e.g., the
    coefficients of a linear model), the goal of recursive feature elimination
    (RFE) is to select features by recursively considering smaller and smaller
    sets of features. First, the estimator is trained on the initial set of
    features and the importance of each feature is obtained either through a
    ``coef_`` attribute or through a ``feature_importances_`` attribute.
    Then, the least important features are pruned from current set of features.
    That procedure is recursively repeated on the pruned set until the desired
    number of features to select is eventually reached.

    Read more in the :ref:`User Guide <rfe>`.

    Parameters
    ----------
    estimator : object
        A supervised learning estimator with a ``fit`` method that provides
        information about feature importance either through a ``coef_``
        attribute or through a ``feature_importances_`` attribute.

    n_features_to_select : int or None (default=None)
        The number of features to select. If `None`, half of the features
        are selected.

    step : int or float, optional (default=1)
        If greater than or equal to 1, then ``step`` corresponds to the
        (integer) number of features to remove at each iteration.
        If within (0.0, 1.0), then ``step`` corresponds to the percentage
        (rounded down) of features to remove at each iteration.

    verbose : int, (default=0)
        Controls verbosity of output.

    Attributes
    ----------
    n_features_ : int
        The number of selected features.

    support_ : array of shape [n_features]
        The mask of selected features.

    ranking_ : array of shape [n_features]
        The feature ranking, such that ``ranking_[i]`` corresponds to the
        ranking position of the i-th feature. Selected (i.e., estimated
        best) features are assigned rank 1.

    estimator_ : object
        The external estimator fit on the reduced dataset.

    Examples
    --------
    The following example shows how to retrieve the 5 right informative
    features in the Friedman #1 dataset.

    >>> from sklearn.datasets import make_friedman1
    >>> from sklearn.feature_selection import RFE
    >>> from sklearn.svm import SVR
    >>> X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
    >>> estimator = SVR(kernel="linear")
    >>> selector = RFE(estimator, 5, step=1)
    >>> selector = selector.fit(X, y)
    >>> selector.support_ # doctest: +NORMALIZE_WHITESPACE
    array([ True,  True,  True,  True,  True, False, False, False, False,
           False])
    >>> selector.ranking_
    array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

    See also
    --------
    RFECV : Recursive feature elimination with built-in cross-validated
        selection of the best number of features

    References
    ----------

    .. [1] Guyon, I., Weston, J., Barnhill, S., & Vapnik, V., "Gene selection
           for cancer classification using support vector machines",
           Mach. Learn., 46(1-3), 389--422, 2002.
    """
    def __init__(self, estimator, n_features_to_select=None, step=1,
                 verbose=0):
        self.estimator = estimator
        self.n_features_to_select = n_features_to_select
        self.step = step
        self.verbose = verbose

    @property
    def _estimator_type(self):
        return self.estimator._estimator_type

    def fit(self, X, y):
        """Fit the RFE model and then the underlying estimator on the selected
           features.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            The training input samples.

        y : array-like, shape = [n_samples]
            The target values.
        """
        return self._fit(X, y)

    def _fit(self, X, y, step_score=None):
        # Parameter step_score controls the calculation of self.scores_
        # step_score is not exposed to users
        # and is used when implementing RFECV
        # self.scores_ will not be calculated when calling _fit through fit

        X, y = check_X_y(X, y, "csc", force_all_finite=False)  # patched
        # Initialization
        n_features = X.shape[1]
        if self.n_features_to_select is None:
            n_features_to_select = n_features // 2
        else:
            n_features_to_select = self.n_features_to_select

        if 0.0 < self.step < 1.0:
            step = int(max(1, self.step * n_features))
        else:
            step = int(self.step)
        if step <= 0:
            raise ValueError("Step must be >0")

        support_ = np.ones(n_features, dtype=np.bool)
        ranking_ = np.ones(n_features, dtype=np.int)

        if step_score:
            self.scores_ = []

        # Elimination
        while np.sum(support_) > n_features_to_select:
            # Remaining features
            features = np.arange(n_features)[support_]

            # Rank the remaining features
            estimator = clone(self.estimator)
            if self.verbose > 0:
                print("Fitting estimator with %d features." % np.sum(support_))

            estimator.fit(X[:, features], y)

            # Get coefs
            if hasattr(estimator, 'coef_'):
                coefs = estimator.coef_
            else:
                coefs = getattr(estimator, 'feature_importances_', None)
            if coefs is None:
                raise RuntimeError('The classifier does not expose '
                                   '"coef_" or "feature_importances_" '
                                   'attributes')

            # Get ranks
            if coefs.ndim > 1:
                ranks = np.argsort(safe_sqr(coefs).sum(axis=0))
            else:
                ranks = np.argsort(safe_sqr(coefs))

            # for sparse case ranks is matrix
            ranks = np.ravel(ranks)

            # Eliminate the worse features
            threshold = min(step, np.sum(support_) - n_features_to_select)

            # Compute step score on the previous selection iteration
            # because 'estimator' must use features
            # that have not been eliminated yet
            if step_score:
                self.scores_.append(step_score(estimator, features))
            support_[features[ranks][:threshold]] = False
            ranking_[np.logical_not(support_)] += 1

        # Set final attributes
        features = np.arange(n_features)[support_]
        self.estimator_ = clone(self.estimator)
        self.estimator_.fit(X[:, features], y)

        # Compute step score when only n_features_to_select features left
        if step_score:
            self.scores_.append(step_score(self.estimator_, features))
        self.n_features_ = support_.sum()
        self.support_ = support_
        self.ranking_ = ranking_

        return self

    @if_delegate_has_method(delegate='estimator')
    def predict(self, X):
        """Reduce X to the selected features and then predict using the
           underlying estimator.

        Parameters
        ----------
        X : array of shape [n_samples, n_features]
            The input samples.

        Returns
        -------
        y : array of shape [n_samples]
            The predicted target values.
        """
        check_is_fitted(self, 'estimator_')
        return self.estimator_.predict(self.transform(X))

    @if_delegate_has_method(delegate='estimator')
    def score(self, X, y):
        """Reduce X to the selected features and then return the score of the
           underlying estimator.

        Parameters
        ----------
        X : array of shape [n_samples, n_features]
            The input samples.

        y : array of shape [n_samples]
            The target values.
        """
        check_is_fitted(self, 'estimator_')
        return self.estimator_.score(self.transform(X), y)

    def _get_support_mask(self):
        check_is_fitted(self, 'support_')
        return self.support_

    @if_delegate_has_method(delegate='estimator')
    def decision_function(self, X):
        """Compute the decision function of ``X``.

        Parameters
        ----------
        X : array-like or sparse matrix, shape = [n_samples, n_features]
            The input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csr_matrix``.

        Returns
        -------
        score : array, shape = [n_samples, n_classes] or [n_samples]
            The decision function of the input samples. The order of the
            classes corresponds to that in the attribute `classes_`.
            Regression and binary classification produce an array of shape
            [n_samples].
        """
        check_is_fitted(self, 'estimator_')
        return self.estimator_.decision_function(self.transform(X))

    @if_delegate_has_method(delegate='estimator')
    def predict_proba(self, X):
        """Predict class probabilities for X.

        Parameters
        ----------
        X : array-like or sparse matrix, shape = [n_samples, n_features]
            The input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csr_matrix``.

        Returns
        -------
        p : array of shape = [n_samples, n_classes]
            The class probabilities of the input samples. The order of the
            classes corresponds to that in the attribute `classes_`.
        """
        check_is_fitted(self, 'estimator_')
        return self.estimator_.predict_proba(self.transform(X))

    @if_delegate_has_method(delegate='estimator')
    def predict_log_proba(self, X):
        """Predict class log-probabilities for X.

        Parameters
        ----------
        X : array of shape [n_samples, n_features]
            The input samples.

        Returns
        -------
        p : array of shape = [n_samples, n_classes]
            The class log-probabilities of the input samples. The order of the
            classes corresponds to that in the attribute `classes_`.
        """
        check_is_fitted(self, 'estimator_')
        return self.estimator_.predict_log_proba(self.transform(X))
    
class RFECV(RFE, MetaEstimatorMixin):
    """Feature ranking with recursive feature elimination and cross-validated
    selection of the best number of features.

    Read more in the :ref:`User Guide <rfe>`.

    Parameters
    ----------
    estimator : object
        A supervised learning estimator with a ``fit`` method that provides
        information about feature importance either through a ``coef_``
        attribute or through a ``feature_importances_`` attribute.

    step : int or float, optional (default=1)
        If greater than or equal to 1, then ``step`` corresponds to the
        (integer) number of features to remove at each iteration.
        If within (0.0, 1.0), then ``step`` corresponds to the percentage
        (rounded down) of features to remove at each iteration.
        Note that the last iteration may remove fewer than ``step`` features in
        order to reach ``min_features_to_select``.

    min_features_to_select : int, (default=1)
        The minimum number of features to be selected. This number of features
        will always be scored, even if the difference between the original
        feature count and ``min_features_to_select`` isn't divisible by
        ``step``.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

        - None, to use the default 3-fold cross-validation,
        - integer, to specify the number of folds.
        - An object to be used as a cross-validation generator.
        - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`sklearn.model_selection.StratifiedKFold` is used. If the
        estimator is a classifier or if ``y`` is neither binary nor multiclass,
        :class:`sklearn.model_selection.KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validation strategies that can be used here.

        .. versionchanged:: 0.20
            ``cv`` default value of None will change from 3-fold to 5-fold
            in v0.22.

    scoring : string, callable or None, optional, (default=None)
        A string (see model evaluation documentation) or
        a scorer callable object / function with signature
        ``scorer(estimator, X, y)``.

    verbose : int, (default=0)
        Controls verbosity of output.

    n_jobs : int or None, optional (default=None)
        Number of cores to run in parallel while fitting across folds.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    Attributes
    ----------
    n_features_ : int
        The number of selected features with cross-validation.

    support_ : array of shape [n_features]
        The mask of selected features.

    ranking_ : array of shape [n_features]
        The feature ranking, such that `ranking_[i]`
        corresponds to the ranking
        position of the i-th feature.
        Selected (i.e., estimated best)
        features are assigned rank 1.

    grid_scores_ : array of shape [n_subsets_of_features]
        The cross-validation scores such that
        ``grid_scores_[i]`` corresponds to
        the CV score of the i-th subset of features.

    estimator_ : object
        The external estimator fit on the reduced dataset.

    Notes
    -----
    The size of ``grid_scores_`` is equal to
    ``ceil((n_features - min_features_to_select) / step) + 1``,
    where step is the number of features removed at each iteration.

    Examples
    --------
    The following example shows how to retrieve the a-priori not known 5
    informative features in the Friedman #1 dataset.

    >>> from sklearn.datasets import make_friedman1
    >>> from sklearn.feature_selection import RFECV
    >>> from sklearn.svm import SVR
    >>> X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
    >>> estimator = SVR(kernel="linear")
    >>> selector = RFECV(estimator, step=1, cv=5)
    >>> selector = selector.fit(X, y)
    >>> selector.support_ # doctest: +NORMALIZE_WHITESPACE
    array([ True,  True,  True,  True,  True, False, False, False, False,
           False])
    >>> selector.ranking_
    array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

    See also
    --------
    RFE : Recursive feature elimination

    References
    ----------

    .. [1] Guyon, I., Weston, J., Barnhill, S., & Vapnik, V., "Gene selection
           for cancer classification using support vector machines",
           Mach. Learn., 46(1-3), 389--422, 2002.
    """
    def __init__(self, estimator, step=1, min_features_to_select=1, cv='warn',
                 scoring=None, verbose=0, n_jobs=None):
        self.estimator = estimator
        self.step = step
        self.cv = cv
        self.scoring = scoring
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.min_features_to_select = min_features_to_select

    def transform(self, X):
        """Reduce X to the selected features.

        Parameters
        ----------
        X : array of shape [n_samples, n_features]
            The input samples.

        Returns
        -------
        X_r : array of shape [n_samples, n_selected_features]
            The input samples with only the selected features.
        """
        X = check_array(X, dtype=None, accept_sparse='csr', force_all_finite=False)
        mask = self.get_support()
        if not mask.any():
            warn("No features were selected: either the data is"
                 " too noisy or the selection test too strict.",
                 UserWarning)
            return np.empty(0).reshape((X.shape[0], 0))
        if len(mask) != X.shape[1]:
            raise ValueError("X has a different shape than during fitting.")
        return X[:, safe_mask(X, mask)]

    def fit(self, X, y, groups=None):
        """Fit the RFE model and automatically tune the number of selected
           features.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vector, where `n_samples` is the number of samples and
            `n_features` is the total number of features.

        y : array-like, shape = [n_samples]
            Target values (integers for classification, real numbers for
            regression).

        groups : array-like, shape = [n_samples], optional
            Group labels for the samples used while splitting the dataset into
            train/test set.
        """
        X, y = check_X_y(X, y, "csr", force_all_finite=False)

        # Initialization
        cv = check_cv(self.cv, y, is_classifier(self.estimator))
        scorer = check_scoring(self.estimator, scoring=self.scoring)
        n_features = X.shape[1]

        if 0.0 < self.step < 1.0:
            step = int(max(1, self.step * n_features))
        else:
            step = int(self.step)
        if step <= 0:
            raise ValueError("Step must be >0")

        # Build an RFE object, which will evaluate and score each possible
        # feature count, down to self.min_features_to_select
        rfe = RFE(estimator=self.estimator,
                  n_features_to_select=self.min_features_to_select,
                  step=self.step, verbose=self.verbose)

        # Determine the number of subsets of features by fitting across
        # the train folds and choosing the "features_to_select" parameter
        # that gives the least averaged error across all folds.

        # Note that joblib raises a non-picklable error for bound methods
        # even if n_jobs is set to 1 with the default multiprocessing
        # backend.
        # This branching is done so that to
        # make sure that user code that sets n_jobs to 1
        # and provides bound methods as scorers is not broken with the
        # addition of n_jobs parameter in version 0.18.

        if effective_n_jobs(self.n_jobs) == 1:
            parallel, func = list, _rfe_single_fit
        else:
            parallel = Parallel(n_jobs=self.n_jobs)
            func = delayed(_rfe_single_fit)

        scores = parallel(
            func(rfe, self.estimator, X, y, train, test, scorer)
            for train, test in cv.split(X, y, groups))

        scores = np.sum(scores, axis=0)
        scores_rev = scores[::-1]
        argmax_idx = len(scores) - np.argmax(scores_rev) - 1
        n_features_to_select = max(
            n_features - (argmax_idx * step),
            self.min_features_to_select)

        # Re-execute an elimination with best_k over the whole set
        rfe = RFE(estimator=self.estimator,
                  n_features_to_select=n_features_to_select, step=self.step,
                  verbose=self.verbose)

        rfe.fit(X, y)

        # Set final attributes
        self.support_ = rfe.support_
        self.n_features_ = rfe.n_features_
        self.ranking_ = rfe.ranking_
        self.estimator_ = clone(self.estimator)
        self.estimator_.fit(self.transform(X), y)

        # Fixing a normalization error, n is equal to get_n_splits(X, y) - 1
        # here, the scores are normalized by get_n_splits(X, y)
        self.grid_scores_ = scores[::-1] / cv.get_n_splits(X, y, groups)
        return self

We will use a terrible training set to demonstrate how this works:

In [ ]:
sample_data = pd.read_csv('../input/training_set_metadata.csv')

In [ ]:
sample_data.head()

In [ ]:
X = sample_data.iloc[:, 1:-1].copy()
y = sample_data['target']
# manually sabotaging the data:
X.loc[X['hostgal_specz'] == 0, 'hostgal_specz'] = np.inf

In [ ]:
X.head()

In [ ]:
X.shape, y.shape

In [ ]:
clf_params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 14,
        'metric': 'multi_logloss',
        'learning_rate': 0.01,
        'n_estimators': 500,
        'verbose': -1,
        'max_depth': 3,
        'importance_type': 'gain',
        'seed': 1111,
        'n_jobs': -1,
    }

In [ ]:
clf = lgb.LGBMClassifier(**clf_params)

In [ ]:
rfe = RFE(clf, step=1, n_features_to_select=1, verbose=1)

In [ ]:
rfe.fit(X, y)

In [ ]:
# feature rankings
{c: r for c, r in zip(X.columns, rfe.ranking_)}

We see that after the patching, RFE works with NaN and Inf just fine.

## 2. Passing parameters to fit() when SkLearn won't let you

Now that you have RFE working, you may want to let it use proper sample weights so your selected features will give you better performance in your final model. However, SkLearn sometimes stubbornly holds onto the simplicity of its API and would not allow you to pass any extra parameters to the call. If you want to do the following:

In [ ]:
w = y.value_counts()
sample_weight_map = {i: np.sum(w) / w[i] for i in w.index}

In [ ]:
rfe.fit(X, y, sample_weight=y.map(sample_weight_map))

it just won't happen. It sometimes actually makes sense, for example with CV estimators, each time the base estimator's `fit()` is called, the X and y might be different (depending on folds), and the sample_weight required may be different, so it does not make sense to pass the same weights to all of these method calls. But here, we really want to pass the same parameter to all of the `clf.fit()` calls. While we cannot change the SkLearn API, we can change the `clf.fit()` method to pass the sample weights automatically when SkLearn calls it. We need to insert something between `rfe.fit()` and SkLearn calling `clf.fit()` to smuggle in the `sample_weight` parameter. One seemingly easy solution is:

In [ ]:
from functools import partial
clf = lgb.LGBMClassifier(**clf_params)
clf.fit = partial(clf.fit, sample_weight=y.map(sample_weight_map))
rfe = RFE(clf, step=1, n_features_to_select=1, verbose=1)
rfe.fit(X, y)

In [ ]:
{c: r for c, r in zip(X.columns, rfe.ranking_)}

It appears to be working, but it is really deceiving. RFE uses `clone()` under the hood to make copies of the base estimator. Let us see if our modification survives the cloning:

In [ ]:
clf = lgb.LGBMClassifier(**clf_params)
clf.fit = partial(clf.fit, sample_weight=y.map(sample_weight_map))
str(clf.fit)[:100]

In [ ]:
str(clone(clf).fit)[:100]

We see that in the cloned classifier, the method reverts to the original. Seems we have to modify the class for this to work. So can we do the same trick to the base class?

In [ ]:
class NewLGBMClassifier(lgb.LGBMClassifier):
    # override
    def fit(self, X, y):
        return super().fit(X, y, sample_weight=pd.Series(y).map(sample_weight_map))
    # ^^^ sklearn actually converts y to numpy here already

In [ ]:
clf = NewLGBMClassifier(**clf_params)
rfe = RFE(clf, step=1, n_features_to_select=1, verbose=1)
rfe.fit(X, y)

In [ ]:
{c: r for c, r in zip(X.columns, rfe.ranking_)}

We get a slightly different rankings, suggesting that something is different this time. The weights are actually being applied now. If we look at the final estimator from RFE:

In [ ]:
import inspect
inspect.signature(rfe.estimator_.fit)

In [ ]:
inspect.signature(lgb.LGBMClassifier.fit)

We see that all the cloned estimators from RFE are the modified version now.

As a bonus note, what if we really only want to modify that one instance? Does the original trick work? The code below will give you an error, indicating that the modifications are indeed applied:
```
clf = lgb.LGBMClassifier(**clf_params)
clf.fit = partial(clf.fit, sample_weight=y.map(sample_weight_map))
clf.fit(X.head(100), y.head(100))
```
However, if you really want a modified instance that can accept variable X, y and apply weights to them, you will need a different trick. We may attempt the following:

In [ ]:
clf = lgb.LGBMClassifier(**clf_params)

def fit_override(self, X, y):
    return clf.fit(X, y, sample_weight=y.map(sample_weight_map))

clf.fit = fit_override
clf.fit(X.head(100), y.head(100))

But what we are really doing is assigning an ordinary function to the attribute `fit` of `clf`. It does not know it is supposed to be a method of `clf`. We should register the function as a method:

In [ ]:
clf = lgb.LGBMClassifier(**clf_params)

def fit_override(self, X, y):
    return self.fit(X, y, sample_weight=y.map(sample_weight_map))
# or clf.fit(X, y, sample_weight=y.map(sample_weight_map))

clf.fit = types.MethodType(fit_override, clf)
clf.fit(X.head(100), y.head(100))

But why is it still reporting an error? Because we are accidentally creating a recursion. When we call `clf.fit()`, it calls `fit_override` instead, then `fit_override` calls `clf.fit`, but `clf.fit` is `fit_override` now... So we are trying to pass `sample_weight` to `fit_override` which should not accept additional parameters other than X, y, hence the error. We can fix with the following trick:

In [ ]:
clf = lgb.LGBMClassifier(**clf_params)

def get_override():
    fit = clf.fit
    # use closure to 'freeze' clf.fit
    def fit_override(self, X, y):
        return fit(X, y, sample_weight=y.map(sample_weight_map))
    return fit_override

clf.fit = types.MethodType(get_override(), clf)
clf.fit(X.head(100), y.head(100))

We see that this code is able to run, so the method freezing trick works! With this trick, you can do all sorts of method modding by inserting arbitrary code between the method call and the original method.